#  Intro to Hardware Security and Trust Final Project

## Import Libraries

In [147]:
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.cluster import KMeans,DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix
from sklearn.metrics import roc_curve

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import random
import os
from statistics import mean


## Dataset Splitting Function

In [148]:
xlsx_path = 'C:\\Users\patri\\git\\HSTFinalProject\\Classifier-for-Hardware-Trojan-Detection\\ROFreq\\ROFreq\\Chip'
csv_path ='C:\\Users\patri\\git\HSTFinalProject\Classifier-for-Hardware-Trojan-Detection\ROFreq_C\Chip'

for i in np.arange(1,34):
    data_xls = pd.read_excel('{}{}.xlsx'.format(xlsx_path,i), dtype=str, index_col=None)
    data_xls.to_csv('{}{}.csv'.format(csv_path,i), encoding='utf-8', index=False)

In [149]:
def csv_read(chip_num,type,csv_path):
    data= []
    data_list= []
    label_list = np.zeros(25)
    label_list[0] = 1
    label_list[-1] = 1
    with open('{}{}.csv'.format(csv_path,chip_num)) as csvfile:
        csv_reader = csv.reader(csvfile)
   
        for row in csv_reader:
            # rounding
            row = [round(float(num[0:12]),3) for num in row]     
            data_list.append(row)   

        if type == 'TI':
            data.append(data_list[0])
            data.append(data_list[-1])
            label = [1,1]
        elif type == 'TF':
            data = data_list[1:-1]
            label = label_list[1:-1]
        else:
            data = data_list
            label = label_list 
                  
        
    return data,label

In [150]:
def train_test_split(num_sel,case_sel,csv_path):
    chips_num = np.arange(1,34)
    indices = range(33)
    # randomly select #num chips
    sel = random.sample(indices,num_sel)
    # split tain and test data
    train_chips = chips_num[sel]
    test_chips = np.delete(chips_num,sel)
    print('Training chips num: ',train_chips)
    temp = []

    for i in range(int(num_sel/2)):
        temp.extend(['TF','TI'])
 
    type_sel_1 = random.sample(temp,num_sel)
    type_sel_3 = random.choices(['TI','TF'],k=num_sel)
    
    if case_sel == 1:
        print('Type selection:', type_sel_1)
        print('TI:',type_sel_1.count('TI'))
        print('TF:',type_sel_1.count('TF'))

    else:
        print('Type selection:', type_sel_3)
        print('TI:',type_sel_3.count('TI'))
        print('TF:',type_sel_3.count('TF'))

    cnt = 0
    train_data = []
    train_label = []

    for chip_num in train_chips:
        if case_sel == 1:
            data,label = csv_read(chip_num,type_sel_1[cnt],csv_path)
        elif case_sel ==3:
            data,label = csv_read(chip_num,type_sel_3[cnt],csv_path)
        train_data.extend(data)
        train_label.extend(label)
        cnt += 1

    test_data = []
    test_label = []
    for chip_num in test_chips:
        data,label = csv_read(chip_num,'ALL',csv_path)
        test_data.extend(data)
        test_label.extend(label)
    
    return train_data, train_label, test_data, test_label

##  CASE 1

### Case 1 Datasets

In [151]:
# arg1 => number of training samples
# arg2 => number of case (enter 1 or 3 only)
train24_data, train24_label, test24_data, test24_label = train_test_split(24,1,csv_path)
train12_data, train12_label, test12_data, test12_label = train_test_split(12,1,csv_path)
train6_data, train6_label, test6_data, test6_label = train_test_split(6,1,csv_path)

Training chips num:  [25 12 26 29 21 22 24 15 16  8  3  7 23 10 28 32  1 14 13 30 20 27 19 33]
Type selection: ['TI', 'TI', 'TI', 'TF', 'TI', 'TI', 'TI', 'TF', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TF', 'TI', 'TF', 'TF', 'TF', 'TF', 'TF', 'TI', 'TI', 'TF']
TI: 12
TF: 12
Training chips num:  [ 4 27 25 26 12 18 33  3 29  9 31  1]
Type selection: ['TI', 'TI', 'TI', 'TF', 'TI', 'TI', 'TI', 'TF', 'TF', 'TF', 'TF', 'TF']
TI: 6
TF: 6
Training chips num:  [ 1  8  3  2 23 11]
Type selection: ['TF', 'TF', 'TI', 'TI', 'TI', 'TF']
TI: 3
TF: 3


In [152]:
pd.DataFrame(train24_data)

,0,1,2,3,4,5,6,7
0,186.0,199.2,216.0,229.6,219.2,221.6,213.2,246.8
1,185.6,198.8,216.0,229.6,218.8,221.6,213.2,246.4
2,184.0,196.0,217.2,227.6,217.2,224.8,215.2,246.8
3,184.0,196.0,217.2,227.6,217.2,224.8,215.2,246.8
4,184.4,197.6,215.6,226.8,217.2,220.0,212.8,244.4
...,...,...,...,...,...,...,...,...
295,158.0,168.4,184.4,194.0,185.2,189.2,181.6,208.0
296,151.2,161.6,177.2,185.6,178.0,181.6,173.6,199.6
297,145.2,155.6,170.0,179.2,171.2,174.8,167.2,192.0
298,140.0,149.6,163.6,172.4,164.4,168.0,161.2,184.8


In [153]:
# 24 Training - Create Dataframe objects and compute correlations
train24_data_df = pd.DataFrame(train24_data)
train24_label_s = pd.Series(train24_label)
train24_data_df.corrwith(train24_label_s).sort_values(ascending=False)

0    0.378748
1    0.368480
5    0.363674
2    0.361671
6    0.359963
7    0.358587
4    0.358117
3    0.354083
dtype: float64

In [154]:
# 12 Training - Create Dataframe objects and compute correlations
train12_data_df = pd.DataFrame(train12_data)
train12_label_s = pd.Series(train12_label)
train12_data_df.corrwith(train12_label_s).sort_values(ascending=False)

7    0.461197
6    0.455739
5    0.453359
4    0.451546
0    0.446819
3    0.444685
1    0.444467
2    0.443684
dtype: float64

In [155]:
# 6 Training - Create Dataframe objects and compute correlations
train6_data_df = pd.DataFrame(train6_data)
train6_label_s = pd.Series(train6_label)
train6_data_df.corrwith(train6_label_s).sort_values(ascending=False)

4    0.365808
1    0.357429
3    0.347645
7    0.339286
5    0.331204
0    0.321546
6    0.308553
2    0.305096
dtype: float64

Correlation between features seems to be about equal for predictions with training and test splits

### Tuning KNeighbors Classifier

In [156]:
# Creating KNeighborsClassifier Pipeline
KneighC = Pipeline([('scaler',StandardScaler()),('knc', KNeighborsClassifier())])

# Finding the best parameters for KNeighborsClassifier

## Create parameter grid for gridsearch algorithm
knc_param_Grid=  { 'knc__n_neighbors': np.arange(2,10,1),
                  'knc__leaf_size': np.arange(2,30,2),
                 }

## Create Grid search
knc_grid_search = GridSearchCV(KneighC,
                               param_grid=knc_param_Grid,
                               scoring='accuracy',
                               refit=True,
                               cv=5,
                               verbose=1)

ii=0
### Fit to training data
KNC_trainscores=[]
KNC_testscores=[]

tpr_train_scores = []
fpr_train_scores = []
tpr_scores = []
fpr_scores = []

while ii < 20:

    # Splits for each dataset
    train24_data, train24_label, test24_data, test24_label = train_test_split(24,1,csv_path)
    train12_data, train12_label, test12_data, test12_label = train_test_split(12,1,csv_path)
    train6_data, train6_label, test6_data, test6_label = train_test_split(6,1,csv_path)
    
    train24_data = pd.DataFrame(train24_data)
    test24_data = pd.DataFrame(test24_data)
    
    train12_data = pd.DataFrame(train12_data)
    test12_data = pd.DataFrame(test12_data)
    
    train6_data = pd.DataFrame(train6_data)
    test6_data = pd.DataFrame(test6_data)
    

    # Create lists of training sets to parse
    training_sets = [train24_data, train12_data, train6_data]
    training_labels = [train24_label, train12_label, train6_label]
    
    # Create lists of test sets to parse
    test_sets = [test24_data, test12_data, test6_data]
    test_labels =[test24_label, test12_label, test6_label]
    
    Sets = ["-----24 Sample Training Set-----","-----12 Sample Training Set-----", "-----6 Sample Training Set-----"]
    t_sets = ["-----24 Sample Test Set-----","-----12 Sample Test Set-----", "-----6 Sample Test Set-----"]
    
    #Iterater
    ii = ii + 1
    
    # Loop responsible for parsing training sets and training labels
    for (a,i,j,m,n) in zip(Sets, training_sets,training_labels, test_sets, test_labels):
        print("\n",a,"\n")
        knc_grid_search.fit(i,j)
        ### Print Best parameters
        print("\n",knc_grid_search.best_params_)
        ## Saving the tuned model
        model = knc_grid_search.best_estimator_
        print("Best Accuracy: ",knc_grid_search.best_score_)
        KNC_trainscores+= [knc_grid_search.best_score_]
        y_pred = model.predict(m)
        KNC_testscores += [accuracy_score(n,y_pred)]
        
        # Calculate Average TPR and Average FPR for training set
        tn, fp, fn, tp = confusion_matrix(j, model.predict(i)).ravel()
        tpr_train = tp/(tp+fn)
        fpr_train = fp/(fp+tn)

        # Create list of TPR and FPR for train
        tpr_train_scores += [tpr_train]
        fpr_train_scores += [fpr_train]

        # Calculate Average TPR and Average FPR for test set
        tn, fp, fn, tp = confusion_matrix(n, y_pred).ravel()
        tpr_test = tp/(tp+fn)
        fpr_test = fp/(fp+tn)
        
        # Create list of TPR and FPR
        tpr_scores += [tpr_test]
        fpr_scores += [fpr_test]

Training chips num:  [11 15 16  5 30  2 32 17  8 10 33  9 29 14 23 26 12 19 20  7  6 21 28  3]
Type selection: ['TF', 'TI', 'TI', 'TF', 'TF', 'TI', 'TI', 'TF', 'TF', 'TF', 'TF', 'TI', 'TF', 'TI', 'TI', 'TF', 'TF', 'TI', 'TI', 'TI', 'TI', 'TI', 'TF', 'TF']
TI: 12
TF: 12
Training chips num:  [24 18 20 29 25  3 22 31  1 10 13 23]
Type selection: ['TF', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TF', 'TF', 'TI', 'TI']
TI: 6
TF: 6
Training chips num:  [26  1 22 29 20 12]
Type selection: ['TI', 'TI', 'TI', 'TF', 'TF', 'TF']
TI: 3
TF: 3

 -----24 Sample Training Set----- 

Fitting 5 folds for each of 112 candidates, totalling 560 fits

 {'knc__leaf_size': 2, 'knc__n_neighbors': 3}
Best Accuracy:  0.9633333333333333

 -----12 Sample Training Set----- 

Fitting 5 folds for each of 112 candidates, totalling 560 fits

 {'knc__leaf_size': 2, 'knc__n_neighbors': 7}
Best Accuracy:  0.9800000000000001

 -----6 Sample Training Set----- 

Fitting 5 folds for each of 112 candidates, totalling 560 fits



### Results for KneighborsClassifier

In [157]:
# ----- Accuracy scores for test and training sets -----
# Slicing training scores into individual lists
KNC_24_train_scores = KNC_trainscores[0::3]
KNC_12_train_scores= KNC_trainscores[1::3]
KNC_6_train_scores = KNC_trainscores[2::3]

# Slicing test scores into individual lists
KNC_24_test_scores = KNC_testscores[0::3]
KNC_12_test_scores = KNC_testscores[1::3]
KNC_6_test_scores = KNC_testscores[2::3]

print("---------- Kneighbors Accuracy Scores----------")

# Accuracy scores for test and training sets
train24_avg = mean(KNC_24_train_scores)
test24_avg = mean(KNC_24_test_scores)
print("TRAIN24 Average Accuracy: ", train24_avg)
print("Test24 Average Accruacy: " , test24_avg)

# Accuracy scores for test and training sets
train12_avg = mean(KNC_12_train_scores)
test12_avg = mean(KNC_12_test_scores)
print("Train12 Average Accuracy: ", train12_avg)
print("Test12 Average Accruacy: " , test12_avg)

# Accuracy Scores for test and training sets
train6_avg = mean(KNC_6_train_scores)
test6_avg = mean(KNC_6_test_scores)
print("TRAIN6 Average Accuracy: ", train6_avg)
print("Test6 Average Accruacy: " , test6_avg)

# ----- Average TPR and Average FPR -----
# TRAINING SET
KNC_24train_TPR = tpr_train_scores[0::3]
KNC_12train_TPR = tpr_train_scores[1::3]
KNC_6train_TPR = tpr_train_scores[2::3]
KNC_24train_FPR = fpr_train_scores[0::3]
KNC_12train_FPR = fpr_train_scores[1::3]
KNC_6train_FPR = fpr_train_scores[2::3]


# TEST SET
KNC_24test_TPR = tpr_scores[0::3]
KNC_12test_TPR = tpr_scores[1::3]
KNC_6test_TPR = tpr_scores[2::3]
KNC_24test_FPR = fpr_scores[0::3]
KNC_12test_FPR = fpr_scores[1::3]
KNC_6test_FPR = fpr_scores[2::3]

print("\n","---------- KNC Average TPR----------")
TPR_train24 = mean(KNC_24train_TPR)
TPR_test24 = mean(KNC_24test_TPR)
print("TRAIN24 Average TPR: ", TPR_train24)
print("Test24 Average TPR: " , TPR_test24)
TPR_train12 = mean(KNC_12train_TPR)
TPR_test12 = mean(KNC_12test_TPR)
print("Train12 Average TPR: ", TPR_train12)
print("Test12 Average TPR: " , TPR_test12)
TPR_train6 = mean(KNC_6train_TPR)
TPR_test6 = mean(KNC_6test_TPR)
print("TRAIN6 Average TPR: ", TPR_train6)
print("Test6 Average TPR: " , TPR_test6)

print("\n","---------- KNC Average FPR----------")
FPR_train24 = mean(KNC_24train_FPR)
FPR_test24 = mean(KNC_24test_FPR)
print("TRAIN24 Average FPR: ", FPR_train24)
print("Test24 Average FPR: " , FPR_test24)
FPR_train12 = mean(KNC_12train_FPR)
FPR_test12 = mean(KNC_12test_FPR)
print("Train12 Average FPR: ", FPR_train12)
print("Test12 Average FPR: " , FPR_test12)
FPR_train6 = mean(KNC_6train_FPR)
FPR_test6 = mean(KNC_6test_FPR)
print("TRAIN6 Average FPR: ", FPR_train6)
print("Test6 Average FPR: " , FPR_test6)


---------- Kneighbors Accuracy Scores----------
TRAIN24 Average Accuracy:  0.9448333333333333
Test24 Average Accruacy:  0.9231111111111111
Train12 Average Accuracy:  0.9520000000000001
Test12 Average Accruacy:  0.9220952380952381
TRAIN6 Average Accuracy:  0.9513333333333334
Test6 Average Accruacy:  0.9171111111111111

 ---------- KNC Average TPR----------
TRAIN24 Average TPR:  0.80625
Test24 Average TPR:  0.55
Train12 Average TPR:  0.9
Test12 Average TPR:  0.5357142857142857
TRAIN6 Average TPR:  0.725
Test6 Average TPR:  0.4759259259259259

 ---------- KNC Average FPR----------
TRAIN24 Average FPR:  0.008876811594202899
Test24 Average FPR:  0.044444444444444446
Train12 Average FPR:  0.003985507246376811
Test12 Average FPR:  0.0443064182194617
TRAIN6 Average FPR:  0.0007246376811594203
Test6 Average FPR:  0.044524959742351045


### Tuning Logistic Regression

In [158]:
# Create Logisitic Regression Model
Logreg = Pipeline([('scaler',StandardScaler()),('logreg',LogisticRegression(random_state=42))])
Logreg

# Finding the best parameters for Logistic Regression

## Create Parameter Grid
logreg_param_grid = {'logreg__C': [0.001,0.01,0.1,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
                     'logreg__tol': [0.0001, 0.001, 0.01]
                    }

## Create Log Reg Gridsearch
logreg_gridsearch = GridSearchCV(Logreg,
                                 param_grid=logreg_param_grid,
                                 scoring='accuracy',
                                 refit=True,
                                 cv=5,
                                 verbose=1)

### Fit to training data
logreg_trainscores=[]
logreg_testscores=[]

# tpr and fpr scores
tpr_scores = []
fpr_scores = []

tpr_train_scores = []
fpr_train_scores = []
tpr_test_scores = []
fpr_test_scores = []

ii=0
while ii < 20:

    # Splits for each dataset
    train24_data, train24_label, test24_data, test24_label = train_test_split(24,1,csv_path)
    train12_data, train12_label, test12_data, test12_label = train_test_split(12,1,csv_path)
    train6_data, train6_label, test6_data, test6_label = train_test_split(6,1,csv_path)
    
    # Create pandas dataframes and series
    train24_data = pd.DataFrame(train24_data)
    test24_data = pd.DataFrame(test24_data)
    
    train12_data = pd.DataFrame(train12_data)
    test12_data = pd.DataFrame(test12_data)
    
    train6_data = pd.DataFrame(train6_data)
    test6_data = pd.DataFrame(test6_data)
    
    # Create lists of training sets to parse
    training_sets = [train24_data, train12_data, train6_data]
    training_labels = [train24_label, train12_label, train6_label]
    
    # Create lists of test sets to parse
    test_sets = [test24_data, test12_data, test6_data]
    test_labels =[test24_label, test12_label, test6_label]
    
    Sets = ["-----24 Sample Training Set-----","-----12 Sample Training Set-----", "-----6 Sample Training Set-----"]
    t_sets = ["-----24 Sample Test Set-----","-----12 Sample Test Set-----", "-----6 Sample Test Set-----"]
    
    #Iterater
    ii = ii + 1
    
  

    # Loop responsible for parsing training sets and training labels
    for (a,i,j,m,n) in zip(Sets, training_sets,training_labels, test_sets, test_labels):
        print("\n",a,"\n")
        logreg_gridsearch.fit(i,j)
        ### Print Best parameters
        print("\n",logreg_gridsearch.best_params_)
        ## Saving the tuned model
        model = logreg_gridsearch.best_estimator_
        print("Best Accuracy: ",logreg_gridsearch.best_score_)
       
        logreg_trainscores+= [logreg_gridsearch.best_score_]
       
        y_pred = model.predict(m)
        logreg_testscores += [accuracy_score(n,y_pred)]

        # Calculate TPR and FPR on training set
        tp, fn, fp, tn = confusion_matrix(j,model.predict(i)).ravel()
        fpr = fp/(fp+tn)
        tpr = tp/(tp+fn)
        
        tpr_train_scores += [tpr]
        fpr_train_scores += [fpr]
        
        # Calculate FPR and TPR on test set
        tp, fn, fp, tn = confusion_matrix(n,y_pred).ravel()
        fpr_test = fp/(fp+tn)
        tpr_test = tp/(tp+fn)

        tpr_test_scores += [tpr_test]
        fpr_test_scores += [fpr_test]

Training chips num:  [26 14 12 27  4  9 11 31 21 29 23 16  7  8 32  5 33 24  2 28 15  3 25 17]
Type selection: ['TF', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TI', 'TF', 'TI', 'TI', 'TF', 'TI', 'TI', 'TI', 'TF', 'TI', 'TF', 'TF', 'TF', 'TI', 'TI', 'TF', 'TF']
TI: 12
TF: 12
Training chips num:  [ 7  2 16  9  3 32 11 27 26 14 33 30]
Type selection: ['TF', 'TF', 'TI', 'TF', 'TI', 'TI', 'TF', 'TF', 'TI', 'TF', 'TI', 'TI']
TI: 6
TF: 6
Training chips num:  [16  2  6 20 14  8]
Type selection: ['TF', 'TF', 'TI', 'TI', 'TF', 'TI']
TI: 3
TF: 3

 -----24 Sample Training Set----- 

Fitting 5 folds for each of 39 candidates, totalling 195 fits

 {'logreg__C': 0.1, 'logreg__tol': 0.0001}
Best Accuracy:  0.9266666666666667

 -----12 Sample Training Set----- 

Fitting 5 folds for each of 39 candidates, totalling 195 fits

 {'logreg__C': 0.001, 'logreg__tol': 0.0001}
Best Accuracy:  0.9199999999999999

 -----6 Sample Training Set----- 

Fitting 5 folds for each of 39 candidates, totalling 195 fits

 {'logr

### Log Reg Results

In [159]:
# Slicing training scores into individual lists
logreg_24_train_scores = logreg_trainscores[0::3]
logreg_12_train_scores= logreg_trainscores[1::3]
logreg_6_train_scores = logreg_trainscores[2::3]

# Slicing test scores into individual lists
logreg_24_test_scores = logreg_testscores[0::3]
logreg_12_test_scores = logreg_testscores[1::3]
logreg_6_test_scores = logreg_testscores[2::3]

print("---------- Logistic Regression Accuracy Scores----------")

# Accuracy scores for test and training sets
train24_avg = mean(logreg_24_train_scores)
test24_avg = mean(logreg_24_test_scores)
print("TRAIN24 Average Accuracy: ", train24_avg)
print("Test24 Average Accruacy: " , test24_avg)

# Accuracy scores for test and training sets
train12_avg = mean(logreg_12_train_scores)
test12_avg = mean(logreg_12_test_scores)
print("Train12 Average Accuracy: ", train12_avg)
print("Test12 Average Accruacy: " , test12_avg)

# Accuracy Scores for test and training sets
train6_avg = mean(logreg_6_train_scores)
test6_avg = mean(logreg_6_test_scores)
print("TRAIN6 Average Accuracy: ", train6_avg)
print("Test6 Average Accruacy: " , test6_avg)

# Slicing fpr and tpr scores for each set
logreg_24_fpr_train = fpr_train_scores[0::3]
logreg_24_tpr_train = tpr_train_scores[0::3]
logreg_12_fpr_train = fpr_train_scores[1::3]
logreg_12_tpr_train = tpr_train_scores[1::3]
logreg_6_fpr_train = fpr_train_scores[2::3]
logreg_6_tpr_train = tpr_train_scores[2::3]

logreg_24_fpr_test = fpr_test_scores[0::3]
logreg_24_tpr_test = tpr_test_scores[0::3]
logreg_12_fpr_test = fpr_test_scores[1::3]
logreg_12_tpr_test = tpr_test_scores[1::3]
logreg_6_fpr_test = fpr_test_scores[2::3]
logreg_6_tpr_test = tpr_test_scores[2::3]


# Printing Mean score for each set

print("---------- Logistic Regression Mean FPR and TPR Scores----------")

# Mean FPR and TPR scores for training sets
print("Mean FPR for 24 Sample Training Set: ", mean(logreg_24_fpr_train))
print("Mean TPR for 24 Sample Training Set: ", mean(logreg_24_tpr_train))
print("Mean FPR for 12 Sample Test Set: ", mean(logreg_12_fpr_test))
print("Mean TPR for 12 Sample Test Set: ", mean(logreg_12_tpr_test))

print("Mean FPR for 12 Sample Training Set: ", mean(logreg_12_fpr_train))
print("Mean TPR for 12 Sample Training Set: ", mean(logreg_12_tpr_train))
print("Mean FPR for 12 Sample Test Set: ", mean(logreg_12_fpr_test))
print("Mean TPR for 12 Sample Test Set: ", mean(logreg_12_tpr_test))

print("Mean FPR for 6 Sample Training Set: ", mean(logreg_6_fpr_train))
print("Mean TPR for 6 Sample Training Set: ", mean(logreg_6_tpr_train))
print("Mean FPR for 6 Sample Test Set: ", mean(logreg_6_fpr_test))
print("Mean TPR for 6 Sample Test Set: ", mean(logreg_6_tpr_test))




---------- Logistic Regression Accuracy Scores----------
TRAIN24 Average Accuracy:  0.9266666666666665
Test24 Average Accruacy:  0.912888888888889
Train12 Average Accuracy:  0.934
Test12 Average Accruacy:  0.9130476190476191
TRAIN6 Average Accuracy:  0.9506666666666667
Test6 Average Accruacy:  0.9130370370370371
---------- Logistic Regression Mean FPR and TPR Scores----------
Mean FPR for 24 Sample Training Set:  0.8729166666666667
Mean TPR for 24 Sample Training Set:  0.9967391304347826
Mean FPR for 12 Sample Test Set:  0.8178571428571428
Mean TPR for 12 Sample Test Set:  0.9766045548654244
Mean FPR for 12 Sample Training Set:  0.7916666666666666
Mean TPR for 12 Sample Training Set:  0.9981884057971014
Mean FPR for 12 Sample Test Set:  0.8178571428571428
Mean TPR for 12 Sample Test Set:  0.9766045548654244
Mean FPR for 6 Sample Training Set:  0.6166666666666667
Mean TPR for 6 Sample Training Set:  1.0
Mean FPR for 6 Sample Test Set:  0.6953703703703704
Mean TPR for 6 Sample Test Set: 

## CASE 3

### KMeans

In [199]:
# function to derive TPR and FPR
def tpr_fpr(clusters,labels):
    # Create confusion matrix
    cm = confusion_matrix(clusters,labels)
     # Derive TPR and FPR from confusion matrix
    tpr = cm[1,1]/(cm[1,0]+cm[1,1])
    fpr = cm[0,1]/(cm[0,0]+cm[0,1])
    return tpr,fpr

In [196]:
# Create KMeans model

# Kmeans pipeline with scaler and pca



kmeans = Pipeline([('scaler',StandardScaler()),('kmeans',KMeans(random_state=42))])

### kmeans accuracy score lists
kmeans_trainscores=[]
kmeans_testscores=[]

# tpr and fpr scores
tpr_scores = []
fpr_scores = []

tpr_train_scores = []
fpr_train_scores = []
tpr_test_scores = []
fpr_test_scores = []

ii=0
while ii < 20:

    # Splits for each dataset
    train24_data, train24_label, test24_data, test24_label = train_test_split(24,3,csv_path)
    train12_data, train12_label, test12_data, test12_label = train_test_split(12,3,csv_path)
    train6_data, train6_label, test6_data, test6_label = train_test_split(6,3,csv_path)
    
    # Create pandas dataframes and series
    train24_data = pd.DataFrame(train24_data)
    test24_data = pd.DataFrame(test24_data)
    
    train12_data = pd.DataFrame(train12_data)
    test12_data = pd.DataFrame(test12_data)
    
    train6_data = pd.DataFrame(train6_data)
    test6_data = pd.DataFrame(test6_data)
    
    # Create lists of training sets to parse
    training_sets = [train24_data, train12_data, train6_data]
    training_labels = [train24_label, train12_label, train6_label]
    
    # Create lists of test sets to parse
    test_sets = [test24_data, test12_data, test6_data]
    test_labels =[test24_label, test12_label, test6_label]
    
    Sets = ["-----24 Sample Training Set-----","-----12 Sample Training Set-----", "-----6 Sample Training Set-----"]
    t_sets = ["-----24 Sample Test Set-----","-----12 Sample Test Set-----", "-----6 Sample Test Set-----"]
    
    #Iterater
    ii = ii + 1

    # Loop responsible for parsing training sets and training labels
    for (a,i,j,m,n) in zip(Sets, training_sets,training_labels, test_sets, test_labels):
        print("\n",a,"\n")
        j = [int(i) for i in j]
        # Fit the model
        kmeans.fit(i)

        # Get clusters from training and test set
        train_clusters = kmeans.predict(i)
        test_clusters = kmeans.predict(m)

        # Compute accuracy score between training cluster labels and training labels
        train_score = accuracy_score(train_clusters,j)
        
        # Compute accuracy score between test cluster labels and test labels
        test_score = accuracy_score(test_clusters,n)

        # Appeniding to kmeans_train_scores list
        kmeans_trainscores.append(train_score)

        # Appending to kmeans_test_scores list
        kmeans_testscores.append(test_score)

        # derive TPR and TPR from training clusters and training labels
        tpr_train, fpr_train = tpr_fpr(train_clusters,j)
        tpr_train_scores.append(tpr_train)
        fpr_train_scores.append(fpr_train)

        # derive TPR and TPR from test clusters and test labels
        tpr_test, fpr_test = tpr_fpr(test_clusters,n)
        tpr_test_scores.append(tpr_test)
        fpr_test_scores.append(fpr_test)
        
        

Training chips num:  [ 2 25 29 18  6 19 14 13  1 11 27  9 23 32 30 10 33 21 31  3 12 22 15 24]
Type selection: ['TF', 'TF', 'TF', 'TF', 'TI', 'TI', 'TI', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TF', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TI', 'TI', 'TF', 'TF']
TI: 11
TF: 13
Training chips num:  [22 31 27 17 14 21 23 13 20 28 25 10]
Type selection: ['TI', 'TI', 'TF', 'TI', 'TI', 'TF', 'TF', 'TI', 'TF', 'TI', 'TF', 'TF']
TI: 6
TF: 6
Training chips num:  [ 7 12 33 27 14 22]
Type selection: ['TF', 'TI', 'TF', 'TF', 'TI', 'TF']
TI: 2
TF: 4

 -----24 Sample Training Set----- 


 -----12 Sample Training Set----- 


 -----6 Sample Training Set----- 

Training chips num:  [13 24 17 16 33  4  9  5 26  2  3 29 30 23 27 25 12  6 18 21 11 10 28  8]
Type selection: ['TI', 'TI', 'TI', 'TF', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TI', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TF', 'TI', 'TI', 'TF', 'TI', 'TI', 'TF']
TI: 14
TF: 10
Training chips num:  [13 28 25  6 22 30 18 21 23 12 11 20]
Type selection: ['TF', 'T

In [197]:
# Slicing training scores into individual lists
kmeans_24_train_scores = kmeans_trainscores[0::3]
kmeans_12_train_scores = kmeans_trainscores[1::3]
kmeans_6_train_scores = kmeans_trainscores[2::3]

# Slicing test scores into individual lists
kmeans_24_test_scores = kmeans_testscores[0::3]
kmeans_12_test_scores = kmeans_testscores[1::3]
kmeans_6_test_scores = kmeans_testscores[2::3]


print("---------- KMEANS Accuracy Scores----------")

# Accuracy scores for test and training sets
train24_avg = mean(kmeans_24_train_scores)
test24_avg = mean(kmeans_24_test_scores)
print("Train24 Average Accuracy: ", train24_avg)
print("Test24 Average Accruacy: " , test24_avg)

# Accuracy scores for test and training sets
train12_avg = mean(kmeans_12_train_scores)
test12_avg = mean(kmeans_12_test_scores)
print("Train12 Average Accuracy: ", train12_avg)
print("Test12 Average Accruacy: " , test12_avg)

# Accuracy Scores for test and training sets
train6_avg = mean(kmeans_6_train_scores)
test6_avg = mean(kmeans_6_test_scores)
print("Train6 Average Accuracy: ", train6_avg)
print("Test6 Average Accruacy: " , test6_avg)

# Slicing fpr and tpr scores for each set
kmeans_24_fpr = fpr_test_scores[0::3]
kmeans_24_tpr = tpr_test_scores[0::3]
kmeans_12_tpr = tpr_test_scores[1::3]
kmeans_12_fpr = fpr_test_scores[1::3]
kmeans_6_tpr = tpr_test_scores[2::3]
kmeans_6_fpr = fpr_test_scores[2::3]


# Printing Mean score for each set

print("---------- KMEANS Mean FPR and TPR Scores----------")

# Mean FPR and TPR scores for kmeans training set
kmeans_24_fpr_train = mean(fpr_train_scores[0::3])
kmeans_24_tpr_train = mean(tpr_train_scores[0::3])
kmeans_12_fpr_train = mean(fpr_train_scores[1::3])
kmeans_12_tpr_train = mean(tpr_train_scores[1::3])
kmeans_6_fpr_train = mean(fpr_train_scores[2::3])
kmeans_6_tpr_train = mean(tpr_train_scores[2::3])

# Mean FPR and TPR scores for kmeans test set
# Find the average between each roc curve in fpr_test_scores[0::3]
kmeans_24_fpr_test = mean(fpr_test_scores[0::3])
kmeans_24_tpr_test = mean(tpr_test_scores[0::3])
kmeans_12_fpr_test = mean(fpr_test_scores[1::3])
kmeans_12_tpr_test = mean(tpr_test_scores[1::3])
kmeans_6_fpr_test = mean(fpr_test_scores[2::3])
kmeans_6_tpr_test = mean(tpr_test_scores[2::3])

# Printing Mean FPR and TPR scores for each set
print("Kmeans 24 Sample Training Set FPR: ", kmeans_24_fpr_train)
print("Kmeans 24 Sample Training Set TPR: ", kmeans_24_tpr_train)
print("Kmeans 24 Sample Test Set FPR: ", kmeans_24_fpr_test)
print("Kmeans 24 Sample Test Set TPR: ", kmeans_24_tpr_test)
print("Kmeans 12 Sample Training Set FPR: ", kmeans_12_fpr_train)
print("Kmeans 12 Sample Training Set TPR: ", kmeans_12_tpr_train)
print("Kmeans 12 Sample Test Set FPR: ", kmeans_12_fpr_test)
print("Kmeans 12 Sample Test Set TPR: ", kmeans_12_tpr_test)
print("Kmeans 6 Sample Training Set FPR: ", kmeans_6_fpr_train)
print("Kmeans 6 Sample Training Set TPR: ", kmeans_6_tpr_train)
print("Kmeans 6 Sample Test Set FPR: ", kmeans_6_fpr_test)
print("Kmeans 6 Sample Test Set TPR: ", kmeans_6_tpr_test)

---------- KMEANS Accuracy Scores----------
Train24 Average Accuracy:  0.12052028207291934
Test24 Average Accruacy:  0.11466666666666667
Train12 Average Accuracy:  0.1321995462865973
Test12 Average Accruacy:  0.13238095238095238
Train6 Average Accuracy:  0.16320729733229733
Test6 Average Accruacy:  0.14237037037037037
---------- KMEANS Mean FPR and TPR Scores----------
Kmeans 24 Sample Training Set FPR:  0.0865453723088362
Kmeans 24 Sample Training Set TPR:  0.035258517377161444
Kmeans 24 Sample Test Set FPR:  0.10688887824118003
Kmeans 24 Sample Test Set TPR:  0.07915046236280356
Kmeans 12 Sample Training Set FPR:  0.1531818181818182
Kmeans 12 Sample Training Set TPR:  0.06335200746965453
Kmeans 12 Sample Test Set FPR:  0.09054325630242858
Kmeans 12 Sample Test Set TPR:  0.0911126954622486
Kmeans 6 Sample Training Set FPR:  0.12361111111111112
Kmeans 6 Sample Training Set TPR:  0.2
Kmeans 6 Sample Test Set FPR:  0.07079319060362771
Kmeans 6 Sample Test Set TPR:  0.14389085022293976


### DBSCAN

In [200]:
# Create DBSCAN Model
dbscan = Pipeline([('scaler',StandardScaler()),('dbscan', DBSCAN())])
dbscan

Pipeline(steps=[('scaler', StandardScaler()), ('dbscan', DBSCAN())])

In [216]:
# Create DBSCAN


#DBSCAN pipeline
dbscan = Pipeline([('scaler',StandardScaler()),('dbscan', DBSCAN())])


### dbscan accuracy score lists
dbscan_trainscores = []
dbscan_testscores = []

# tpr and fpr scores
tpr_scores = []
fpr_scores = []

tpr_train_scores = []
fpr_train_scores = []
tpr_test_scores = []
fpr_test_scores = []

ii=0
while ii < 20:


	# Splits for each dataset
	train24_data, train24_label, test24_data, test24_label = train_test_split(24,3,csv_path)
	train12_data, train12_label, test12_data, test12_label = train_test_split(12,3,csv_path)
	train6_data, train6_label, test6_data, test6_label = train_test_split(6,3,csv_path)

	# Create pandas dataframes and series
	train24_data = pd.DataFrame(train24_data)
	test24_data = pd.DataFrame(test24_data)
	train12_data = pd.DataFrame(train12_data)
	test12_data = pd.DataFrame(test12_data)
	train6_data = pd.DataFrame(train6_data)
	test6_data = pd.DataFrame(test6_data)

	# Create lists of training sets to parse
	training_sets = [train24_data, train12_data, train6_data]
	training_labels = [train24_label, train12_label, train6_label]

	# Create lists of test sets to parse
	test_sets = [test24_data, test12_data, test6_data]
	test_labels =[test24_label, test12_label, test6_label]
	Sets = ["-----24 Sample Training Set-----","-----12 Sample Training Set-----", "-----6 Sample Training Set-----"]
	t_sets = ["-----24 Sample Test Set-----","-----12 Sample Test Set-----", "-----6 Sample Test Set-----"]

	#Iterater
	ii = ii + 1

	# Loop responsible for parsing training sets and training labels
	for (a,i,j,m,n) in zip(Sets, training_sets,training_labels, test_sets, test_labels): 
		print("\n",a,"\n")
		j = [int(i) for i in j]
		# Fit the model
		dbscan.fit(i)

		# Get clusters from training and test set
		train_clusters = dbscan.named_steps['dbscan'].labels_

		# Compute accuracy score between training cluster labels and training labels
		train_score = accuracy_score(train_clusters,j)

		# Appeniding to kmeans_train_scores list
		dbscan_trainscores.append(train_score)

		# Fit model to test set
		dbscan.fit(m)

		# Compute test clusters
		test_clusters = dbscan.named_steps['dbscan'].labels_

		# Compute accuracy score between test cluster labels and test labels
		test_score = accuracy_score(test_clusters,n)

		# Appending to kmeans_test_scores list
		dbscan_testscores.append(test_score)

		# derive TPR and TPR from training clusters and training labels
		tpr_train, fpr_train = tpr_fpr(train_clusters,j)
		tpr_train_scores.append(tpr_train)
		fpr_train_scores.append(fpr_train)

		# derive TPR and TPR from test clusters and test labels
		tpr_test, fpr_test = tpr_fpr(test_clusters,n)
		tpr_test_scores.append(tpr_test)
		fpr_test_scores.append(fpr_test)

Training chips num:  [ 8  5 21 33 25  3 24 11 32 22 20  2  9 17 29 10 27 28 31 30  4 13  7 16]
Type selection: ['TI', 'TI', 'TI', 'TF', 'TF', 'TF', 'TI', 'TI', 'TF', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TF', 'TF', 'TF']
TI: 11
TF: 13
Training chips num:  [27  1 25 33 16 11 23 17  2 19  6  3]
Type selection: ['TI', 'TF', 'TI', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TF', 'TF', 'TI']
TI: 6
TF: 6
Training chips num:  [ 2 26  9 21 10 23]
Type selection: ['TI', 'TI', 'TF', 'TF', 'TF', 'TI']
TI: 3
TF: 3

 -----24 Sample Training Set----- 


 -----12 Sample Training Set----- 


 -----6 Sample Training Set----- 

Training chips num:  [23 11 28 24 25 26  1  6  8 31  3 17 20 27  9 22 13 19  4 12 14 30 15 18]
Type selection: ['TI', 'TI', 'TI', 'TF', 'TI', 'TI', 'TI', 'TI', 'TI', 'TI', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TF', 'TI', 'TF', 'TI', 'TI', 'TI', 'TI', 'TI']
TI: 18
TF: 6
Training chips num:  [13 28 10 30 31  8 18  1 14  4 25 17]
Type selection: ['TF', 'TI

C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  tpr = cm[1,1]/(cm[1,0]+cm[1,1])
C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr = cm[0,1]/(cm[0,0]+cm[0,1])
C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr = cm[0,1]/(cm[0,0]+cm[0,1])
C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  tpr = cm[1,1]/(cm[1,0]+cm[1,1])



 -----24 Sample Training Set----- 


 -----12 Sample Training Set----- 


 -----6 Sample Training Set----- 

Training chips num:  [ 2 26  9  3  5 16 27 22 12 13 21 14 10 28 24  6 11 33 17  1 29 25 20  4]
Type selection: ['TF', 'TF', 'TI', 'TI', 'TF', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TF', 'TF', 'TF', 'TF', 'TI', 'TF', 'TF', 'TF', 'TI', 'TI', 'TI', 'TI']
TI: 11
TF: 13
Training chips num:  [24 18 12 15 14 29 33 10 22 31  8  1]
Type selection: ['TI', 'TF', 'TI', 'TF', 'TI', 'TF', 'TF', 'TF', 'TF', 'TI', 'TF', 'TF']
TI: 4
TF: 8
Training chips num:  [12 26  5  4 24 22]
Type selection: ['TF', 'TF', 'TF', 'TF', 'TF', 'TI']
TI: 1
TF: 5

 -----24 Sample Training Set----- 


 -----12 Sample Training Set----- 


 -----6 Sample Training Set----- 

Training chips num:  [18 23 11  1 22 17 26 31  9 10  7 25 12  5  6 16  4 19 30 14 20 29 13  2]
Type selection: ['TI', 'TF', 'TI', 'TF', 'TF', 'TF', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TF', 'TI', 'TI', 'TF', 'TI', 'TI', 'TF', 'TI', 'TF', 'TF', 

C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr = cm[0,1]/(cm[0,0]+cm[0,1])
C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  tpr = cm[1,1]/(cm[1,0]+cm[1,1])


Training chips num:  [ 1 12 19 14  4  8]
Type selection: ['TF', 'TI', 'TF', 'TI', 'TI', 'TF']
TI: 3
TF: 3

 -----24 Sample Training Set----- 


 -----12 Sample Training Set----- 


 -----6 Sample Training Set----- 

Training chips num:  [24  7 11 33 19 14 31 10  2 26  6 17  1  9 27 30 23 32 29 12  3 13 28  4]
Type selection: ['TI', 'TF', 'TF', 'TI', 'TF', 'TF', 'TF', 'TF', 'TF', 'TI', 'TI', 'TI', 'TI', 'TF', 'TF', 'TF', 'TF', 'TI', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI']
TI: 10
TF: 14
Training chips num:  [ 4  9 14 29 17 19 15 32 16  5  6 25]
Type selection: ['TF', 'TF', 'TI', 'TI', 'TF', 'TF', 'TF', 'TF', 'TF', 'TF', 'TF', 'TI']
TI: 3
TF: 9
Training chips num:  [ 5  2 24 22 30 26]
Type selection: ['TF', 'TI', 'TF', 'TI', 'TF', 'TI']
TI: 3
TF: 3

 -----24 Sample Training Set----- 


 -----12 Sample Training Set----- 


 -----6 Sample Training Set----- 

Training chips num:  [27 15  7 19 10 33  6 16 18 31 29 17  9 20 14  5 25  4 13 30 32 11  2  8]
Type selection: ['TF', 'TF', 'TI', 'TF', 'T

C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr = cm[0,1]/(cm[0,0]+cm[0,1])
C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr = cm[0,1]/(cm[0,0]+cm[0,1])



 -----6 Sample Training Set----- 

Training chips num:  [ 5  6 18  7 21 13 23 28 27  9 20 25 22 15 29 32 11 16 14 31 24 33 30  4]
Type selection: ['TI', 'TI', 'TF', 'TI', 'TI', 'TI', 'TI', 'TF', 'TF', 'TF', 'TI', 'TF', 'TI', 'TF', 'TI', 'TF', 'TF', 'TF', 'TF', 'TF', 'TF', 'TI', 'TI', 'TF']
TI: 11
TF: 13
Training chips num:  [18 12 10 29  1  9 30 11  2  5 17 20]
Type selection: ['TF', 'TF', 'TF', 'TI', 'TF', 'TF', 'TI', 'TI', 'TF', 'TF', 'TF', 'TF']
TI: 3
TF: 9
Training chips num:  [32  9  6 29 25 30]
Type selection: ['TI', 'TF', 'TF', 'TF', 'TF', 'TI']
TI: 2
TF: 4

 -----24 Sample Training Set----- 


 -----12 Sample Training Set----- 


 -----6 Sample Training Set----- 

Training chips num:  [28 13  6  2 11  4 12 23 14 24 22 31 17  8 20 15 19 30 32 26  3 25  5 10]
Type selection: ['TF', 'TF', 'TI', 'TF', 'TI', 'TI', 'TI', 'TF', 'TF', 'TF', 'TF', 'TI', 'TF', 'TF', 'TF', 'TI', 'TF', 'TF', 'TF', 'TI', 'TF', 'TI', 'TI', 'TF']
TI: 9
TF: 15
Training chips num:  [32 21  3 20  7  2 12 30 19 

C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr = cm[0,1]/(cm[0,0]+cm[0,1])
C:\Users\patri\AppData\Local\Temp\ipykernel_22968\217114975.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  tpr = cm[1,1]/(cm[1,0]+cm[1,1])


In [220]:
# Slicing training scores into individual lists
dbscan_24_train_scores = dbscan_trainscores[0::3]
dbscan_12_train_scores = dbscan_trainscores[1::3]
dbscan_6_train_scores = dbscan_trainscores[2::3]

# Slicing test scores into individual lists
dbscan_24_test_scores = dbscan_testscores[0::3]
dbscan_12_test_scores = dbscan_testscores[1::3]
dbscan_6_test_scores = dbscan_testscores[2::3]


print("---------- DBSCAN Accuracy Scores----------")

# Accuracy scores for test and training sets
train24_avg = mean(dbscan_24_train_scores)
test24_avg = mean(dbscan_24_test_scores)
print("Train24 Average Accuracy: ", train24_avg)
print("Test24 Average Accruacy: " , test24_avg)

# Accuracy scores for test and training sets
train12_avg = mean(dbscan_12_train_scores)
test12_avg = mean(dbscan_12_test_scores)
print("Train12 Average Accuracy: ", train12_avg)
print("Test12 Average Accruacy: " , test12_avg)

# Accuracy Scores for test and training sets
train6_avg = mean(dbscan_6_train_scores)
test6_avg = mean(dbscan_6_test_scores)
print("Train6 Average Accuracy: ", train6_avg)
print("Test6 Average Accruacy: " , test6_avg)

# Slicing fpr and tpr scores for each set
dbscan_24_fpr = fpr_test_scores[0::3]
dbscan_24_tpr = tpr_test_scores[0::3]
dbscan_12_tpr = tpr_test_scores[1::3]
dbscan_12_fpr = fpr_test_scores[1::3]
dbscan_6_tpr = tpr_test_scores[2::3]
dbscan_6_fpr = fpr_test_scores[2::3]


# Printing Mean score for each set

print("---------- DBSCAN Mean FPR and TPR Scores----------")
# Mean FPR and TPR scores for DBSCAN training set
dbscan_24_fpr_train = mean(fpr_train_scores[0::3])
dbscan_24_tpr_train = mean(tpr_train_scores[0::3])
dbscan_12_fpr_train = mean(fpr_train_scores[1::3])
dbscan_12_tpr_train = mean(tpr_train_scores[1::3])
dbscan_6_fpr_train = mean(fpr_train_scores[2::3])
dbscan_6_tpr_train = mean(tpr_train_scores[2::3])

# print the mean scores
print("Mean FPR for 24 Sample Training Set: ", dbscan_24_fpr_train)
print("Mean TPR for 24 Sample Training Set: ", dbscan_24_tpr_train)
print("Mean FPR for 12 Sample Training Set: ", dbscan_12_fpr_train)
print("Mean TPR for 12 Sample Training Set: ", dbscan_12_tpr_train)
print("Mean FPR for 6 Sample Training Set: ", dbscan_6_fpr_train)
print("Mean TPR for 6 Sample Training Set: ", dbscan_6_tpr_train)




---------- DBSCAN Accuracy Scores----------
Train24 Average Accuracy:  0.9028110337113915
Test24 Average Accruacy:  0.8764444444444445
Train12 Average Accuracy:  0.8288179801815106
Test12 Average Accruacy:  0.9160952380952381
Train6 Average Accuracy:  0.5793929287298852
Test6 Average Accruacy:  0.9153333333333333
---------- DBSCAN Mean FPR and TPR Scores----------
Mean FPR for 24 Sample Training Set:  nan
Mean TPR for 24 Sample Training Set:  1.0
Mean FPR for 12 Sample Training Set:  nan
Mean TPR for 12 Sample Training Set:  0.95
Mean FPR for 6 Sample Training Set:  0.9523809523809523
Mean TPR for 6 Sample Training Set:  nan
